# User-Defined Types: Data Classes

In [75]:
import datetime as dt
from collections import namedtuple
from pprint import pprint
from enum import auto, Enum
from fractions import Fraction
from dataclasses import dataclass
from typing import Set
from copy import deepcopy



## Data Classes in Action


In [28]:
# A Fraction represents the relationship between that numerator and denominator

Fraction(numerator=3, denominator=5)


Fraction(3, 5)

In [29]:
# The same fraction can be represented with a dataclass, as follows

@dataclass
class MyFraction:
    numerator: int = 3
    denominator: int = 5


In [30]:
# Consider an automated soup maker. If we need to group my soup ingredients together, 
# we can use dataclasses, as follows:

class ImperialMeasure(Enum):
    TEASPOON = auto()
    TABLESPOON = auto()
    CUP = auto()

class Broth(Enum):
    VEGETABLE = auto()
    CHICKEN = auto()
    BEEF = auto()
    FISH = auto()

@dataclass(frozen=True) # Indecates immutable dataclass
class Ingredient:
    """
    Ingredients added into the broth
    """
    name: str
    amount: float = 1
    units: ImperialMeasure = ImperialMeasure.CUP

@dataclass
class Recipe:
    broth: Broth
    aromatics: set[Ingredient]
    vegetables: set[Ingredient]
    meats: set[Ingredient]
    starches: set[Ingredient]
    garnishes: set[Ingredient]
    time_to_cook: dt.timedelta


In [37]:
# We can instantiate the dataclasses as follows:

pepper = Ingredient("Pepper", 1, ImperialMeasure.TABLESPOON)
garlic = Ingredient("Garlic", 2, ImperialMeasure.TEASPOON)
carrots = Ingredient("Carrots", .25, ImperialMeasure.CUP)
celery = Ingredient("Celery", .25, ImperialMeasure.CUP)
onions = Ingredient("Onions", .25, ImperialMeasure.CUP)
parsley = Ingredient("Parsley", 2, ImperialMeasure.TABLESPOON)
noodles = Ingredient("Noodles", 1.5, ImperialMeasure.CUP)
chicken = Ingredient("Chicken", 1.5, ImperialMeasure.CUP)

def create_soup():
    return Recipe(
        broth=Broth.CHICKEN,
        aromatics={pepper, garlic},
        vegetables={celery, onions, carrots},
        meats={chicken},
        starches={noodles},
        garnishes={parsley},
        time_to_cook=dt.timedelta(minutes=60),
    )

chicken_noodle_soup = create_soup()


In [32]:
# We can also get and set individual fields

chicken_noodle_soup.broth


<Broth.CHICKEN: 2>

In [33]:
chicken_noodle_soup.garnishes.add(pepper)


In [34]:
# If we want to make any soup vegetarian by substituting vegetable broth and removing any meats,
# We can add that behavior as methods of the dataclass, as follows

@dataclass
class Recipe:
    broth: Broth
    aromatics: set[Ingredient]
    vegetables: set[Ingredient]
    meats: set[Ingredient]
    starches: set[Ingredient]
    garnishes: set[Ingredient]
    time_to_cook: dt.timedelta

    def make_vegetarian(self):
        self.meats.clear()
        self.broth = Broth.VEGETABLE

    def get_ingredient_names(self):
        ingredients = (
            self.aromatics |
            self.vegetables |
            self.meats |
            self.starches |
            self.garnishes
        )
        
        return (
            {i.name for i in ingredients} |
            {self.broth.name.capitalize() + " broth"}
        )


In [39]:
# We can instantiate the last dataclass as follows

chicken_noodle_soup = create_soup()

# Make a deep copy so that changing one soup does not change the original
veggie_noodle_soup = deepcopy(chicken_noodle_soup) 
veggie_noodle_soup.make_vegetarian()
veggie_noodle_soup.get_ingredient_names()


{'Carrots',
 'Celery',
 'Garlic',
 'Noodles',
 'Onions',
 'Parsley',
 'Pepper',
 'Vegetable broth'}

## Usage of dataclasses

### String Conversion


In [43]:
# Data classes define the __str__ and __repr__ functions automatically

pprint(str(chicken_noodle_soup))


("Recipe(broth=<Broth.CHICKEN: 2>, aromatics={Ingredient(name='Garlic', "
 "amount=2, units=<ImperialMeasure.TEASPOON: 1>), Ingredient(name='Pepper', "
 'amount=1, units=<ImperialMeasure.TABLESPOON: 2>)}, '
 "vegetables={Ingredient(name='Carrots', amount=0.25, "
 "units=<ImperialMeasure.CUP: 3>), Ingredient(name='Celery', amount=0.25, "
 "units=<ImperialMeasure.CUP: 3>), Ingredient(name='Onions', amount=0.25, "
 "units=<ImperialMeasure.CUP: 3>)}, meats={Ingredient(name='Chicken', "
 'amount=1.5, units=<ImperialMeasure.CUP: 3>)}, '
 "starches={Ingredient(name='Noodles', amount=1.5, units=<ImperialMeasure.CUP: "
 "3>)}, garnishes={Ingredient(name='Parsley', amount=2, "
 'units=<ImperialMeasure.TABLESPOON: 2>)}, '
 'time_to_cook=datetime.timedelta(seconds=3600))')


### Equality

In [50]:
# To be able to test equality (==, !=) between two data classes, specify eq=True 
# when defining the dataclass, as follows

@dataclass(eq=True)
class MyClass:
    field1: str
    field2: int

obj1 = MyClass('a', 1)
obj2 = MyClass('a', 1)
obj3 = MyClass('b', 2)


In [51]:
obj1 == obj2


True

In [52]:
obj1 != obj3


True

### Relational Comparison


In [53]:
# To be able to define relational comparison (<, >, <=, >=), specify eq=True and order=True 
# when defining the dataclass, as follows

@dataclass(eq=True, order=True)
class NutritionInformation:
    calories: int
    fat: int
    carbohydrates: int

nutritionals = [
    NutritionInformation(calories=100, fat=1, carbohydrates=3),
    NutritionInformation(calories=50, fat=6, carbohydrates=4),
    NutritionInformation(calories=125, fat=12, carbohydrates=3)
]


In [54]:
sorted(nutritionals)


[NutritionInformation(calories=50, fat=6, carbohydrates=4),
 NutritionInformation(calories=100, fat=1, carbohydrates=3),
 NutritionInformation(calories=125, fat=12, carbohydrates=3)]

In [60]:
# We can also implementt custom relational comparisons.
# For example, to sort NutritionInformation first by fat,then carbohydrates, and then calories, 
# we can implement the dataclass as follows

@dataclass(eq=True)
class NutritionInformation:
    calories: int
    fat: int
    carbohydrates: int

    def __lt__(self, rhs) -> bool:
        return (
            (self.fat, self.carbohydrates, self.calories) < 
            (rhs.fat, rhs.carbohydrates, rhs.calories)
        )

    def __le__(self, rhs) -> bool:
        return self < rhs or self == rhs

    def __gt__(self, rhs) -> bool:
        return not self <= rhs

    def __ge__(self, rhs) -> bool:
        return not self < rhs

nutritionals2 = [
    NutritionInformation(calories=100, fat=1, carbohydrates=3),
    NutritionInformation(calories=50, fat=6, carbohydrates=4),
    NutritionInformation(calories=125, fat=12, carbohydrates=3)
]


In [59]:
sorted(nutritionals2)

[NutritionInformation(calories=100, fat=1, carbohydrates=3),
 NutritionInformation(calories=50, fat=6, carbohydrates=4),
 NutritionInformation(calories=125, fat=12, carbohydrates=3)]

### Immutability

In [62]:
# To enforce immutability in a dataclass, specify frozen=True, as follows

@dataclass(frozen=True)
class Recipe:
    broth: Broth
    aromatics: Set[Ingredient]
    vegetables: Set[Ingredient]
    meats: Set[Ingredient]
    starches: Set[Ingredient]
    garnishes: Set[Ingredient]
    time_to_cook: dt.timedelta


Caveats around dataclass immutability

In [63]:
# First, dataclass immutability means the fields in the dataclass are immutable, 
# not the variable containing the dataclass itself

# Assume that Recipe is immutable because frozen was set to true in the decorator

soup = Recipe(
    broth=Broth.CHICKEN,
    aromatics={pepper, garlic},
    vegetables={celery, onions, carrots},
    meats={chicken},
    starches={noodles},
    garnishes={parsley},
    time_to_cook=dt.timedelta(minutes=60)
)


In [65]:
soup.broth = Broth.VEGETABLE # This is illegal


FrozenInstanceError: cannot assign to field 'broth'

In [72]:
# But this is perfectly legal

soup = Recipe(
    broth=Broth.CHICKEN,
    aromatics=set(),
    vegetables=set(),
    meats=set(),
    starches=set(),
    garnishes=set(),
    time_to_cook=dt.timedelta(seconds=3600)
)


In [73]:
# Second, a frozen dataclass only prevents its members from being set. 
# If the members are mutable, you are still able to modify their values. 

soup.aromatics


set()

In [74]:
soup.aromatics.add(Ingredient("Garlic"))
soup.aromatics

{Ingredient(name='Garlic', amount=1, units=<ImperialMeasure.CUP: 3>)}